In [2]:
import torch
from sklearn.model_selection import train_test_split
import yaml
import argparse
import shutil
from collections import namedtuple
import os
import datetime


# Custom Libraries
from utils.data_generator import DataGenerator
from utils.agent import TrainModel
from utils.helper import to_python_native, gen_experiment_name, set_seed, save_model_state
from utils.pennylane.model import Qkernel

In [4]:
# === Backend Configuration ===
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.xpu.is_available():
    device = torch.device("xpu")
else:
    device = torch.device("cpu")

set_seed(42)
device

device(type='mps')

In [10]:
data_generator = DataGenerator(
        dataset_name= 'iris',
        file_path=None,
    )

training_data, training_labels, testing_data, testing_labels = data_generator.generate_dataset()
training_data = torch.tensor(training_data.to_numpy(), dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(testing_data.to_numpy(), dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(training_labels.to_numpy(), dtype=torch.int)
testing_labels = torch.tensor(testing_labels.to_numpy(), dtype=torch.int)

In [11]:
print(training_data.size())
print(testing_data.size())

torch.Size([120, 4])
torch.Size([30, 4])


In [12]:
kernel = Qkernel(
        device='lightning.qubit',
        n_qubits=4,
        trainable=True,
        input_scaling=True,
        data_reuploading=True,
        ansatz='embedding_paper',
        ansatz_layers=5
    )

In [13]:
agent = TrainModel(
        kernel=kernel,
        training_data=training_data,
        training_labels=training_labels,
        testing_data=testing_data,
        testing_labels=testing_labels,
        optimizer='gd',
        lr= 0.1,
        mclr=0.01,
        cclr=0.01,
        epochs=100,
        train_method='ccka',
        target_accuracy=0.95,
        get_alignment_every=100,
        validate_every_epoch=1,
        base_path='../',
        lambda_kao=0.001,
        lambda_co=0.001,
        clusters=8,
        use_kmeans=True
    )

Epochs:  10


In [14]:
before_metrics = agent.evaluate_test(testing_data, testing_labels, 'before')
before_metrics

proto_labels: [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
len(subcentroids): 24
{1: [Parameter containing:
tensor([-2.1443, -0.1870, -2.7004, -2.8798], requires_grad=True), Parameter containing:
tensor([-1.4711,  1.5334, -2.5331, -2.5806], requires_grad=True), Parameter containing:
tensor([-2.6180,  1.0472, -3.1416, -2.8798], requires_grad=True), Parameter containing:
tensor([-1.2217,  0.5236, -2.3961, -2.8798], requires_grad=True), Parameter containing:
tensor([-2.5482,  0.3142, -2.5878, -2.8274], requires_grad=True), Parameter containing:
tensor([-2.9671, -1.0472, -2.8487, -2.8798], requires_grad=True), Parameter containing:
tensor([-0.9599,  2.5525, -2.7156, -2.8143], requires_grad=True), Parameter containing:
tensor([-1.8088,  0.7140, -2.6478, -2.6894], requires_grad=True)], 2: [Parameter containing:
tensor([-0.7951, -1.8908, -0.0059, -0.4654], requires_grad=True), Parameter containing:
tensor([-0.0349, -0.7330,  0.4366,  0.2094], requires_grad=True), P

{'alignment': tensor(0.9894, grad_fn=<SqueezeBackward0>),
 'executions': None,
 'training_accuracy': 0.9583333333333334,
 'testing_accuracy': 0.9,
 'f1_score': 0.8960127591706538,
 'alignment_arr': [[], []],
 'loss_arr': [],
 'validation_accuracy_arr': []}

In [15]:
 agent.fit_multiclass(training_data, training_labels)

Started Training
proto_labels: [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
len(subcentroids): 24
{1: [Parameter containing:
tensor([-2.1680, -0.1821, -2.7136, -2.8693], requires_grad=True), Parameter containing:
tensor([-1.4655,  1.5383, -2.5326, -2.5765], requires_grad=True), Parameter containing:
tensor([-2.6208,  1.0405, -3.1373, -2.8793], requires_grad=True), Parameter containing:
tensor([-1.2122,  0.5210, -2.3935, -2.8806], requires_grad=True), Parameter containing:
tensor([-2.5410,  0.2971, -2.5874, -2.8289], requires_grad=True), Parameter containing:
tensor([-2.9544, -1.0372, -2.8472, -2.8845], requires_grad=True), Parameter containing:
tensor([-0.9610,  2.5555, -2.7225, -2.8129], requires_grad=True), Parameter containing:
tensor([-1.8300,  0.7007, -2.6529, -2.6955], requires_grad=True)], 2: [Parameter containing:
tensor([-7.8759e-01, -1.9022e+00,  9.4661e-04, -4.7104e-01],
       requires_grad=True), Parameter containing:
tensor([-0.0031, -0.7115,  

(Qkernel(),
 [Parameter containing:
  tensor([[1.4663, 1.7172, 1.1197, 0.9288],
          [1.2609, 1.4307, 0.9162, 1.3737],
          [0.6506, 1.2660, 0.9270, 0.8982],
          [1.1446, 1.2703, 1.8529, 1.1672],
          [0.6968, 0.9845, 0.9074, 0.6695]], requires_grad=True),
  Parameter containing:
  tensor([[ 2.8387,  1.0656, -2.3263,  2.1354],
          [-0.0715, -1.4058, -2.3036, -2.8417],
          [ 1.5708, -0.4017,  1.3773, -2.4902],
          [ 1.7550, -1.6753, -0.6853,  1.2045],
          [ 1.5274,  2.3885,  2.6952, -2.3460]], requires_grad=True),
  Parameter containing:
  tensor([[ 0.5696, -2.1991, -0.7476, -2.4326],
          [ 3.2298, -2.6811,  1.8434,  0.9145],
          [ 0.6375, -2.0358,  2.4693,  0.1595],
          [ 0.5421, -2.8267,  1.4111,  0.1581],
          [-1.3554,  2.5413,  0.6596, -1.6369]], requires_grad=True)],
 ParameterList(
     (0): Parameter containing: [torch.float32 of size 4]
     (1): Parameter containing: [torch.float32 of size 4]
     (2): Paramet

In [16]:
after_metrics = agent.evaluate_test(testing_data, testing_labels, 'after')
after_metrics

proto_labels: [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
len(subcentroids): 24
{1: [Parameter containing:
tensor([-2.1848, -0.2035, -2.7414, -2.8351], requires_grad=True), Parameter containing:
tensor([-1.4022,  1.6127, -2.5392, -2.5765], requires_grad=True), Parameter containing:
tensor([-2.5817,  1.0398, -3.1230, -2.8876], requires_grad=True), Parameter containing:
tensor([-1.1955,  0.5382, -2.3828, -2.8919], requires_grad=True), Parameter containing:
tensor([-2.5276,  0.3136, -2.6333, -2.8140], requires_grad=True), Parameter containing:
tensor([-2.9349, -1.0094, -2.8357, -2.8618], requires_grad=True), Parameter containing:
tensor([-0.8765,  2.5483, -2.7580, -2.8000], requires_grad=True), Parameter containing:
tensor([-2.0943,  0.5184, -2.7083, -2.7035], requires_grad=True)], 2: [Parameter containing:
tensor([-0.7573, -1.8406, -0.0250, -0.4527], requires_grad=True), Parameter containing:
tensor([ 0.2104, -0.1707,  0.3657,  0.4418], requires_grad=True), P

{'alignment': tensor(0.9167, grad_fn=<SqueezeBackward0>),
 'executions': 7200,
 'training_accuracy': 0.9083333333333333,
 'testing_accuracy': 0.6666666666666666,
 'f1_score': 0.6447619047619048,
 'alignment_arr': [[], []],
 'loss_arr': [],
 'validation_accuracy_arr': []}